In [18]:
import pandas as pd
from tensorflow import keras
import json
import datetime
import numpy as np

In [190]:
input_train = open('5000_payers_input.json')
input_train_arr = json.loads(input_train.read())
input_train.close()
input_train = open('5000_payers_result_train.json')
input_train_result = json.loads(input_train.read())
input_train.close()
input_train = open('5000_payers_result_test.json')
input_test_result = json.loads(input_train.read())
input_train.close()
embedding = open('5000_phones_to_category.json')
phones_to_category = json.loads(embedding.read())
embedding.close()
# print(input_train_arr[1][0][0])

In [199]:
max_dims = max([len(i) for i in input_train_arr])
# print(len(input_train_arr[2]))
for i in range(len(input_train_arr)):
    for j in range(max_dims - len(input_train_arr[i])):
        input_train_arr[i].append([0, 0, 0])
np_input = np.array(input_train_arr)
# np_train = np.array(input_train, dtype=object)
print(np_input.shape)
input_train = np.array(input_train_result)
print(input_train.shape)
input_test = np.array(input_test_result)
# print(input_test.shape)

np_input1 = np_input[:4000]
np_input2 = np_input[4001:]
input_train1 = input_train[:4000]
input_train2 = input_train[4001:]

print(np_input1.shape)
print(input_train1.shape)
print(np_input2.shape)
print(input_train2.shape)

# print(input_train_arr)

(5143, 1747, 3)
(5143,)
(4000, 1747, 3)
(4000,)
(1142, 1747, 3)
(1142,)


In [281]:
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(1747, 3,)),
    keras.layers.Dense(20, activation='softmax'),
    keras.layers.Dense(10, activation='softmax'),
    keras.layers.Dense(5, activation='softmax'),
    keras.layers.Dense(211, activation='softmax')
])

In [282]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=[keras.metrics.TopKCategoricalAccuracy(
    k=3, name="top_k_categorical_accuracy", dtype=None
)])

In [283]:
model.fit(np_input1, input_train1, epochs=50)

Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 5.1866 - top_k_categorical_accuracy: 0.3293
Epoch 2/50
125/125 [==============================] - 0s 2ms/step - loss: 4.8614 - top_k_categorical_accuracy: 1.0000
Epoch 3/50
125/125 [==============================] - 0s 2ms/step - loss: 4.5423 - top_k_categorical_accuracy: 1.0000
Epoch 4/50
125/125 [==============================] - 0s 2ms/step - loss: 4.2278 - top_k_categorical_accuracy: 1.0000
Epoch 5/50
125/125 [==============================] - 0s 2ms/step - loss: 3.9603 - top_k_categorical_accuracy: 1.0000
Epoch 6/50
125/125 [==============================] - 0s 2ms/step - loss: 3.7547 - top_k_categorical_accuracy: 1.0000
Epoch 7/50
125/125 [==============================] - 0s 2ms/step - loss: 3.5990 - top_k_categorical_accuracy: 1.0000
Epoch 8/50
125/125 [==============================] - 0s 2ms/step - loss: 3.4807 - top_k_categorical_accuracy: 1.0000
Epoch 9/50
125/125 [==============================] - 0s

In [249]:
len(phones_to_category)
category_to_phones = []
for i in range(len(phones_to_category)):
    category_to_phones.append({v: k for k, v in phones_to_category[i].items()})

print(category_to_phones)

[{0: '79211031739', 1: '79210623462', 2: '79214692056', 3: '79213357042', 4: '79212109131', 5: '79214749994', 6: '79219635271', 7: '79214691457', 8: '79216860415', 9: '79213357177', 10: '79220691964', 11: '79216859565', 12: '79210243596', 13: '79214695635', 14: '79218286011', 15: '79216859479', 16: '79220729935', 17: '79221033816', 18: '79220660638', 19: '79214695614', 20: '79218199423', 21: '79213360850', 22: '79216859442', 23: '79211032193', 24: '79210716031', 25: '79212110124', 26: '79216860210'}, {0: '79220885615', 1: '79212372416', 2: '79219725622', 3: '79210925737', 4: '79219694244', 5: '79217944459', 6: '79219710207', 7: '79219004124'}, {0: '79221073575', 1: '79215773077', 2: '79215680719', 3: '79221034227', 4: '79216113950', 5: '79215995166'}, {0: '79210158542', 1: '79221033825', 2: '79214748997', 3: '79214748426', 4: '79219636044', 5: '79212044168', 6: '79210243338', 7: '79219635743', 8: '79214749923', 9: '79210126022', 10: '79210157406', 11: '79210838324', 12: '79218432537', 

In [264]:
predictions = model.predict(np_input2)
# print(predictions)
# model.evaluate(np_input2, input_train2)

36/36 [==============================] - 0s 1ms/step


In [284]:
# print(predictions)
count_success = 0
count_fail = 0
for i in range(len(predictions)):

    try:
        # print(i+1, ':', [category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][0]],category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][1]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][2]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][3]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][4]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][5]]], category_to_phones[i][input_train2[i]], end=' result: ')
        if category_to_phones[i][input_train2[i]] in [category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][0]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][1]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][2]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][3]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][4]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][5]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][6]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][7]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][8]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][9]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][10]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][11]]]:
            # print("YES")
            count_success += 1
        else:
            # print('NO')
            count_fail += 1
    except:
        count_fail += 1
        pass

print("SUCCESS RATE TOP12 =", count_success/(count_success+count_fail))

SUCCESS RATE TOP12 = 0.5148861646234676


In [285]:
count_success = 0
count_fail = 0
for i in range(len(predictions)):

    try:
        # print(i+1, ':', [category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][0]],category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][1]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][2]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][3]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][4]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][5]]], category_to_phones[i][input_train2[i]], end=' result: ')
        if category_to_phones[i][input_train2[i]] in [category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][0]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][1]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][2]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][3]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][4]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][5]]]:
            # print("YES")
            count_success += 1
        else:
            # print('NO')
            count_fail += 1
    except:
        count_fail += 1
        pass

print("SUCCESS RATE TOP6 =", count_success/(count_success+count_fail))

SUCCESS RATE TOP6 = 0.5


In [286]:
count_success = 0
count_fail = 0
for i in range(len(predictions)):

    try:
        # print(i+1, ':', [category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][0]],category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][1]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][2]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][3]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][4]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][5]]], category_to_phones[i][input_train2[i]], end=' result: ')
        if category_to_phones[i][input_train2[i]] in [category_to_phones[i][np.argpartition(predictions[i], -3)[-3:][0]], category_to_phones[i][np.argpartition(predictions[i], -3)[-3:][1]], category_to_phones[i][np.argpartition(predictions[i], -3)[-3:][2]]]:
            # print("YES")
            count_success += 1
        else:
            # print('NO')
            count_fail += 1
    except:
        count_fail += 1
        pass

print("SUCCESS RATE TOP3 =", count_success/(count_success+count_fail))

SUCCESS RATE TOP3 = 0.38704028021015763


In [268]:
    ##combine freq and nn

In [287]:
from collections import Counter

success_count = 0
fail_count = 0

only_input_numbers = []
# print(final_array_train)
for i in range(len(np_input2)):
    only_input_numbers.append([np_input2[i][j][2] for j in range(len(np_input2[i]))])
    only_input_numbers[i] = list(dict.fromkeys(only_input_numbers[i])) # remove duplicates
# print(only_input_numbers[0])
for payer in range(len(np_input2)):
# for payer in range(2):
#     print(final_array_train[payer], random.sample(only_input_numbers[payer], 3))
    try:
        # print(Counter(only_input_numbers[payer]).most_common(3))
        nn_sample = [[np.argpartition(predictions[payer], -3)[-3:][0]], [np.argpartition(predictions[payer], -3)[-3:][1]],  [np.argpartition(predictions[payer], -3)[-3:][2]]]
        frequency_sample = [Counter(only_input_numbers[payer]).most_common(3)[i][0] for i in range(3)]
        sample = nn_sample + frequency_sample
        if input_train2[payer] in sample:
            # print(len(only_input_numbers[payer]), final_array_train[payer], random_sample)
            success_count += 1
        else:
            fail_count += 1
    except IndexError as e:
        success_count += 1

print("COMBINED TOP3 SUCCESS RATE =", success_count/(success_count+fail_count))

COMBINED TOP3 SUCCESS RATE = 0.4001751313485114


In [288]:
from collections import Counter

success_count = 0
fail_count = 0

only_input_numbers = []
# print(final_array_train)
for i in range(len(np_input2)):
    only_input_numbers.append([np_input2[i][j][2] for j in range(len(np_input2[i]))])
    only_input_numbers[i] = list(dict.fromkeys(only_input_numbers[i])) # remove duplicates
# print(only_input_numbers[0])
for payer in range(len(np_input2)):
# for payer in range(2):
#     print(final_array_train[payer], random.sample(only_input_numbers[payer], 3))
    try:
        # print(Counter(only_input_numbers[payer]).most_common(3))
        nn_sample = [category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][0]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][1]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][2]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][3]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][4]], category_to_phones[i][np.argpartition(predictions[i], -6)[-6:][5]]]
        frequency_sample = [Counter(only_input_numbers[payer]).most_common(6)[i][0] for i in range(6)]
        sample = nn_sample + frequency_sample
        if input_train2[payer] in sample:
            # print(len(only_input_numbers[payer]), final_array_train[payer], random_sample)
            success_count += 1
        else:
            fail_count += 1
    except IndexError as e:
        success_count += 1

print("COMBINED TOP6 SUCCESS RATE =", success_count/(success_count+fail_count))

COMBINED TOP6 SUCCESS RATE = 0.5542907180385289


In [289]:
from collections import Counter

success_count = 0
fail_count = 0

only_input_numbers = []
# print(final_array_train)
for i in range(len(np_input2)):
    only_input_numbers.append([np_input2[i][j][2] for j in range(len(np_input2[i]))])
    only_input_numbers[i] = list(dict.fromkeys(only_input_numbers[i])) # remove duplicates
# print(only_input_numbers[0])
for payer in range(len(np_input2)):
# for payer in range(2):
#     print(final_array_train[payer], random.sample(only_input_numbers[payer], 3))
    try:
        # print(Counter(only_input_numbers[payer]).most_common(3))
        nn_sample = [category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][0]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][1]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][2]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][3]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][4]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][5]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][6]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][7]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][8]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][9]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][10]], category_to_phones[i][np.argpartition(predictions[i], -12)[-12:][11]]]
        frequency_sample = [Counter(only_input_numbers[payer]).most_common(12)[i][0] for i in range(12)]
        sample = nn_sample + frequency_sample
        if input_train2[payer] in sample:
            # print(len(only_input_numbers[payer]), final_array_train[payer], random_sample)
            success_count += 1
        else:
            fail_count += 1
    except IndexError as e:
        success_count += 1

print("COMBINED TOP12 SUCCESS RATE =", success_count/(success_count+fail_count))

COMBINED TOP12 SUCCESS RATE = 0.7180385288966725
